In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings
warnings.simplefilter("ignore")
gc.collect()

0

# 説明
- ざっくりとした背景
    - df_user から user_item_matrix (pd.Dataframe)を作成する。
    - user_item_matrix には行に user_id, 列に 商品ID、セルには2つの組みに対応するスコアが入る。

    - ※下記リンクの「Step 4: Create User-Movie Matrix」に該当する。
    - https://grabngoinfo.com/recommendation-system-item-based-collaborative-filtering/

- 処理フロー
    - 1. user_item_matrixを作成するためにユーザーID,商品ID,スコアのdictを作成する。
    - 2. 上記のdict から user_item_matrixの作成

- 結論
    - ファイルの最後にある pivot_table で作りたい dataframe できていた・・・


In [5]:
df_user = pd.read_csv("./df_user.csv")
df_user.dtypes

Unnamed: 0     int64
user_id        int64
product_id    object
score          int64
dtype: object

### 1. user_item_matrixを作成するためにユーザーID,商品ID,スコアのdictを作成する。

In [6]:
# ユニークなユーザー、商品IDのリストを作成。pd.Dataframe の index, columns に代入、pd.Datafram の data の作成に使用。
uni_user_id = df_user["user_id"].unique()
uni_prod_id = df_user["product_id"].unique()

# score がある ユーザー、商品ID,スコアをリストにする
user = list(df_user["user_id"].values,)
prod = list(df_user["product_id"].values)
score = list(df_user["score"].values)

In [7]:
dict_score = {}
pre_name = ""

# 一意の user_id, prod_name から score を出せる dict を作成
for user_name, prod_name, score_val in zip(user, prod, score):
    # ユーザーが切り替わった時
    if pre_name != user_name:
        pre_name = user_name
        dict_score[str(user_name)] = {}

    dict_score[str(user_name)][str(prod_name)]= score_val

In [8]:
pre_name = ""
exit_score_data = {}

# score を持つ user_id, product_id の dict を作成。スコアを持っている組か判断。
for user_name, prod_name in zip(user, prod):
    # ユーザーが切り替わった時
    if pre_name != user_name:
        pre_name = user_name
        exit_score_data[user_name] = []
    exit_score_data[user_name].append(str(prod_name))


In [9]:
del user, prod
gc.collect()

0

In [10]:
# user_item_matrix の pd.Dataframe の data に入れる用のリスト。data に入れる前に np.array.reshape で最後に整形する。
score_data = []
counter = 0

for user_id in uni_user_id:
    for prod in uni_prod_id:
        # user_id, product_id がリストの中にあったら（スコアを持っていたら）スコアの値を入れる 
        if prod in exit_score_data[user_id]:
            score_data.append(dict_score[str(user_id)][str(prod)])

        # スコアを持たない組であれば0を入れる
        else:
            score_data.append(0)

### 2. 上記のdict から user_item_matrixの作成

In [11]:
# 縦にユーザーID、横に商品IDのデータフレームを作成。セルにはユーザーIDと商品IDのスコアが入る.
user_item_matrix         = pd.DataFrame(
    data    = np.array(score_data).reshape(len(uni_user_id), len(uni_prod_id)),
    index   = uni_user_id,
    columns = uni_prod_id,
)

user_item_matrix.head()

,00001254_a,00003009_a,00003316_a,00003524_a,00004433_a,00008263_a,00008525_a,00009250_a,00009753_a,00012725_a,...,00001300_a,00010647_a,00008145_a,00009410_a,00001232_a,00006414_a,00013717_a,00010143_a,00001315_a,00001964_a
0,5,5,5,5,10,5,5,10,10,5,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# 保存用
# user_item_matrix.to_csv("./user_item_matrix.csv")

In [13]:
# スコアが入っているdict
dict_score

{'0': {'00001254_a': 5,
  '00003009_a': 5,
  '00003316_a': 5,
  '00003524_a': 5,
  '00004433_a': 10,
  '00008263_a': 5,
  '00008525_a': 5,
  '00009250_a': 10,
  '00009753_a': 10,
  '00012725_a': 5,
  '00013303_a': 5,
  '00014068_a': 5},
 '1': {'00000099_a': 10,
  '00002087_a': 5,
  '00003520_a': 10,
  '00004190_a': 5,
  '00006040_a': 5,
  '00006853_a': 5,
  '00008154_a': 20,
  '00008234_a': 5,
  '00008379_a': 10,
  '00010169_a': 10,
  '00010627_a': 5,
  '00011914_a': 5,
  '00012734_a': 5,
  '00013956_a': 5,
  '00013996_a': 5},
 '2': {'00001418_a': 5,
  '00001653_a': 10,
  '00002183_a': 20,
  '00002826_a': 10,
  '00003936_a': 10,
  '00004638_a': 5,
  '00004816_a': 15,
  '00004929_a': 10,
  '00005472_a': 15,
  '00005559_a': 5,
  '00006003_a': 15,
  '00006497_a': 10,
  '00007036_a': 5,
  '00008763_a': 85,
  '00008916_a': 10,
  '00009275_a': 20,
  '00009388_a': 5,
  '00009816_a': 5,
  '00010377_a': 5,
  '00010585_a': 10,
  '00011876_a': 5,
  '00012097_a': 5,
  '00013249_a': 10,
  '00013488

In [20]:
# 確認用
print(
    user_item_matrix.loc[0, "00001254_a"],
    user_item_matrix.loc[1, "00006040_a"],
    user_item_matrix.loc[1, "00010169_a"],
    # 存在しないデータ -> 0が出力されるはず
    user_item_matrix.loc[1, "00014068_a"],
    sep="\n",
)

5
5
10
0


### pivot_table でできているのでは？

In [25]:
pivot = df_user.pivot_table(index="user_id", columns="product_id", values="score", aggfunc=np.sum, fill_value=0)
pivot.head()

product_id,00000000_a,00000001_a,00000002_a,00000003_a,00000004_a,00000005_a,00000006_a,00000007_a,00000008_a,00000009_a,...,00014113_a,00014114_a,00014115_a,00014116_a,00014117_a,00014118_a,00014119_a,00014120_a,00014121_a,00014122_a
user_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# ごめんなさい、普通に作成できた。。。。
pivot.loc[0, "00001254_a"]

5

In [2]:
import gc
gc.collect()

520